<a href="https://colab.research.google.com/github/Coopyrightdmin/Text-Summarization/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization

based on the following article: https://stackabuse.com/text-summarization-with-nltk-in-python/ with many bugs and not very performant.

In [9]:
import bs4 as bs  
import urllib.request  
import re
import heapq  
from nltk import *

scraped_data = urllib.request.urlopen('https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier')  
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:  
    article_text += p.text
    
# Removing Square Brackets and Extra Spaces
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text)  

# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

sentence_list = nltk.sent_tokenize(article_text)  

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}  
for word in nltk.word_tokenize(formatted_article_text):  
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)  
print(summary)  

L'ombre de « Napoléon le Grand » plane sur de nombreux ouvrages de Balzac, Stendhal, Musset, mais aussi de Dostoïevski, de Tolstoï et de bien d'autres encore. L’agonie de l’Empereur se poursuit, Caulaincourt sort de la pièce pour demander au valet de chambre et au service intérieur de garder le silence. La faveur publique lui permet de rédiger la Constitution de l’an VIII, qui lui donne la réalité des pouvoirs et surtout ne fait pas mention de la souveraineté nationale. Tout en organisant l’Italie en républiques sœurs sur le modèle de la République française, il marche sur l’Autriche et signe seul les préliminaires de paix de Leoben. En conservant et en inscrivant dans le Code tous ces acquis de la Révolution, Bonaparte leur permit de traverser les régimes et rassura une grande partie de la population. Le 5 mars, inquiet de la popularité de Bonaparte, le Directoire le charge de mener l'expédition en Égypte, avec l'arrière-pensée de s'en débarrasser. Les libertés d’expression, de réunio

# TextRank Algorithm
refering to the following article: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
You can download the dataset we’ll be using from [link text](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/tennis_articles_v4.csv)



In [8]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import bs4 as bs  
import urllib.request  

df = ['https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier']

# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
  
sentences = []
for s in df:
  print(s)
  scraped_data = urllib.request.urlopen(s)  
  article = scraped_data.read()
  parsed_article = bs.BeautifulSoup(article,'lxml')
  paragraphs = parsed_article.find_all('p')
  article_text = ""
  for p in paragraphs:  
    article_text += p.text
    
  # Removing Square Brackets and Extra Spaces
  article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
  article_text = re.sub(r'\s+', ' ', article_text)  
  sentences.append(sent_tokenize(article_text))

sentences = [y for x in sentences for y in x] # flatten list

# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)
  
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
      
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier
Lacrosse et son successeur Ernouf réintroduisent progressivement l'esclavage sous la forme d'un travail forcé, puis de l'esclavage lui-même par le biais d'un arrêté de police rural du 22 avril 1803 qui fait référence à certains articles du Code noir, et enfin avec l'adoption du décret du Premier consul, daté du 16 juillet 1802, qui énonce que « La Colonie de la Guadeloupe et Dépendances sera régie, à l’instar de La Martinique, de Ste-Lucie, de Tabago, et des Colonies orientales, par les mêmes loix qui y étoient en vigueur en 1789 ».
Le général noir est gouverneur général de la colonie au nom de la France depuis 1797 et il est suspecté de connivences avec les États-Unis avec lesquels, au mépris du principe de l'Exclusif, il commerce ouvertement depuis que la prospérité est revenue.
La mort de Napoléon résulterait donc de la combinaison d'une hépatite chronique ancienne, d'un ulcère de l'estomac lié au stress évoluant jusqu'à la perforation